In [ ]:
library(Seurat)
library(Matrix)
library(tidyverse)

In [ ]:
source("../../util/sc_preprocess.R")

In [ ]:
# The following RDS files are products from import_DGE.R

store_dir <- "intermediate_dataset/dge/"

dge_base_h1 <- readRDS(paste0(store_dir,"crazy10x_original_heart1_rawdge.rds"))
dge_base_h2 <- readRDS(paste0(store_dir,"crazy10x_original_heart2_rawdge.rds"))
dge_base_n1 <- readRDS(paste0(store_dir,"crazy10x_original_neuron1_rawdge.rds"))
dge_base_n2 <- readRDS(paste0(store_dir,"crazy10x_original_neuron2_rawdge.rds"))

dge_10x_h1 <- readRDS(paste0(store_dir,"crazy10x_to10x_heart1_rawdge.rds"))
dge_10x_h2 <- readRDS(paste0(store_dir,"crazy10x_to10x_heart2_rawdge.rds"))
dge_10x_n1 <- readRDS(paste0(store_dir,"crazy10x_to10x_neuron1_rawdge.rds"))
dge_10x_n2 <- readRDS(paste0(store_dir,"crazy10x_to10x_neuron2_rawdge.rds"))

In [ ]:
bclist_h1  <- read.table("barcode_correspondence_data//FigS4/cb.correspo.heart1",header=F)[,2:3]
bclist_h2  <- read.table("barcode_correspondence_data//FigS4/cb.correspo.heart2",header=F)[,2:3]
bclist_n1  <- read.table("barcode_correspondence_data//FigS4/cb.correspo.neuron1",header=F)[,2:3]
bclist_n2  <- read.table("barcode_correspondence_data//FigS4/cb.correspo.neuron2",header=F)[,2:3]


In [ ]:
dge_10x_h1_new  <- refresh_dge(dge_10x_h1,bclist_h1$V2,bclist_h1$V3,add="-1")
dge_10x_h2_new  <- refresh_dge(dge_10x_h2,bclist_h2$V2,bclist_h2$V3,add="-1")
dge_10x_n1_new  <- refresh_dge(dge_10x_n1,bclist_n1$V2,bclist_n1$V3,add="-1")
dge_10x_n2_new  <- refresh_dge(dge_10x_n2,bclist_n2$V2,bclist_n2$V3,add="-1")

In [ ]:
colnames(dge_10x_h2_new) %>% head

In [ ]:
dge_target_list <- list(list(dge_base_h1,dge_10x_h1_new,"crazy_10x_heart1"),
                        list(dge_base_h2,dge_10x_h2_new,"crazy_10x_heart2"),
                        list(dge_base_n1,dge_10x_n1_new,"crazy_10x_neuron1"),
                        list(dge_base_n2,dge_10x_n2_new,"crazy_10x_neuron2"))

In [ ]:
for(dge_set in dge_target_list){
    dge_base_now <- dge_set[[1]]
    dge_10x_now  <- dge_set[[2]]
    use_cells <- get_cells_used(dge_base_now,list(dge_set[[1]],dge_set[[2]]),percent_mito = 20)
    print(length(use_cells))
    
    #Cell reordering
    dge_base_intersect  <- dge_base_now[,use_cells]
    dge_to10x_intersect  <- dge_10x_now[,use_cells]
    
    #Create Seurat Object
    orig.intersect.seu   <- CreateSeuratObject(dge_base_intersect)
    to10x.intersect.seu  <- CreateSeuratObject(dge_to10x_intersect)
    
    #High-var genes
    high_var_genes <- getVarGenes_untilPCA(orig.intersect.seu,num_varFeatures = 5000)
    orig.intersect.seu   <- getVarGenes_untilPCA(orig.intersect.seu,var_genes=high_var_genes,num_varFeatures = 5000)
    to10x.intersect.seu  <- getVarGenes_untilPCA(to10x.intersect.seu,var_genes=high_var_genes,num_varFeatures = 5000)
    
    #Deimension reduction
    orig.intersect.seu   <- seurat_processing_UMAP(orig.intersect.seu,dimlimit = 20,res = 0.6)
    to10x.intersect.seu  <- seurat_processing_UMAP(to10x.intersect.seu,dimlimit = 20,res = 0.6)
}

In [ ]:
seed=1
for(prefix in c("crazy_10x_heart1","crazy_10x_heart2","crazy_10x_neuron1","crazy_10x_neuron2")){
    print(prefix)
    store_dir <- "seurat_rds/"
    orig.intersect.seu <- readRDS(paste0(store_dir,prefix,"_10x_original_Seurat.rds"))
    to10x.intersect.seu <-readRDS(paste0(store_dir,prefix,"_10x_to_10x_Seurat.rds"))
    seed=seed+1
    draw_umap_crazy10x(seu_base = orig.intersect.seu,
                  seu_to10x = to10x.intersect.seu,
                  col_seed = seed,
                       dot_pt=1,
          w=10,
          h=10,
          is_pdf=F,
          dpi=600,
                  outdir = "figure_out/",
                  outname = paste0(prefix,"_10x"))
    
    # Export CellCellDist
    v.vln.to10x <- get_dist_pcaspace_scatter(orig.intersect.seu,
                                             to10x.intersect.seu,
                                             n_cell_sampling = 50000,
                                             outdir = "figure_out/",
                                             outname = paste0(prefix,"_10x_randUMI"))
    
    v.vln.to10x.scr <- get_dist_pcaspace_scatter(orig.intersect.seu,
                                             to10x.intersect.seu,
                                             n_cell_sampling = 50000,
                                             n_scramble_sampling=100,
                                            scramble = T,
                                             outdir = "figure_out/",
                                             outname = paste0(prefix,"_10x_randUMI_scramble"))
    saveRDS(v.vln.to10x,
            paste0("~/work/barista/paper_data/R_fig_script//intermediate_dataset/CCDist_Apr2021/",prefix,".diff.rds"))
    saveRDS(v.vln.to10x.scr,
            paste0("~/work/barista/paper_data/R_fig_script//intermediate_dataset/CCDist_Apr2021/",prefix,".scr.diff.rds"))
    
}